# Twitter Sentiment Analysis for Coinmarketcap

Get the top 10 cryptocurrencies with Coinmarketcap APIs and analyse the user sentiments with Twitter APIs.
Therefore, we have 2 stepts:
1. Create a function to give you the list of Top10 cryptocurrencies from Coinmarketcap.
2. Create a procedure to get and analyse the sentiment 15 tweets for a cryptocurrency.
3. Use the above to analyse the sentiment for all Top10 cryptocurrencies.

## Step 1: Get Top10 cryptocurrencies from Coinmarketcap

We will use *urllib.request* to read the output of an URL and *json* to understand the API result from the URL.

In [1]:
def GetTop10cryptos():
    # use APIs from coinmarketcap.com to get top10 cryptos (name and website_slug)
    # output: a list of pairs (name and website_slug) for each crypto
    
    # import libraries for Coinmarketcap
    import urllib.request # get the API result from URL
    import json           # handling of JSON API results
    
    # define the Coinmarketcap API URL for top 10 coins ("limit=10")
    url = "https://api.coinmarketcap.com/v2/ticker/?limit=10"

    # open url and read as JSON format the output
    data = json.loads(urllib.request.urlopen(url).read()) # JSON  = dictionary
    
    # initialize an empty list with top crypto to search later in Twitter
    cryptos = []

    # append only (names, website slug) for each crypto in top10
    for x, y in (data['data']).items():
        cryptos.append((y['name'], y['website_slug']))
    
    return cryptos # return list of pairs (name,website_slug) for each crypto

Testing the function: let's get the Top10 cryptos:

In [2]:
print('-> Testing Coinmarketcap API - Top 10 cryptos:')
for crypto in GetTop10cryptos():
    print(crypto[0],' OR ' , crypto[1])

-> Testing Coinmarketcap API - Top 10 cryptos:
Bitcoin  OR  bitcoin
Ethereum  OR  ethereum
XRP  OR  ripple
Bitcoin Cash  OR  bitcoin-cash
EOS  OR  eos
Stellar  OR  stellar
Litecoin  OR  litecoin
Cardano  OR  cardano
Tether  OR  tether
Monero  OR  monero


## Step 2: Get and analyse the sentiment 15 tweets for a crypto

You will use two packages:
* *tweepy* to receive tweets; you will need your API information (consumer key, consumer secret, access token and access secret) from your twitter developer account (https://developer.twitter.com/en/docs/basics/authentication/guides/access-tokens.html).
* *TextBlob* for sentiment analysis.

Let's first design a procedure to save your twitter API data:

In [3]:
def SaveTwitterCredentials(file2save):
    import json 
    # Enter your keys/secrets as strings in the following fields
    credentials = {}  
    credentials['CONSUMER_KEY']    = 'YOUR DATA'  
    credentials['CONSUMER_SECRET'] = 'YOUR DATA'  
    credentials['ACCESS_TOKEN']    = 'YOUR DATA'  
    credentials['ACCESS_SECRET']   = 'YOUR DATA'

    # Save the credentials object to file
    print('-> Saving twitter API credentials',file2save,'...')
    with open(file2save, "w") as file:  
        json.dump(credentials, file)
    print('Done!')
    return

Saving credential for Twitter API:

In [4]:
SaveTwitterCredentials("twitter_credentials.json")

-> Saving twitter API credentials twitter_credentials.json ...
Done!


In [5]:
def GetTweetSentiments(crypto,language='en'):
    # get tweets and analyse sentiment polarity and subjectivity for a specific language
    # INPUTS:
    # - crypto   = (name,website_slug) for a crypto
    # - language = en (default); if can be "es", etc.
    # OUTPUTS:
    # - (tweets mean polarity, tweets mean subjectivity)
    
    from twython import Twython
    from textblob import TextBlob
    import re, json
    import pandas as pd
    
    # Load credentials from json file
    with open("twitter_credentials.json", "r") as file:  
        creds = json.load(file)

    # Instantiate an object for tweets
    python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])
    
    # create search with 2 words: crypto name and website_slug
    keywords = " OR ".join(crypto)
    
    # Create our query
    query = {'q': keywords,  
            'result_type': 'popular',
            'lang': language}

    # create dictionary to retrive tweets info
    dict_ = {'crypto': [], 'date': [], 'text': [], 'polarity': [], 'subjectivity': []}  
    
    # Search tweets for the keywords and estimate sentiments
    # dict_ will will be filled with crypto, date, tweet text, 
    # polarity and subjectivity for each tweet for a specific crypto
    
    for status in python_tweets.search(**query)['statuses']:
        dict_['crypto'].append(keywords)           # a crypto
        dict_['date'].append(status['created_at']) # tweet date
        dict_['text'].append(status['text'])       # tweet text
        
        # Perform Sentiment Analysis on Tweets
        # ****************************************
        # correct encoding
        tweet = status['text'].encode("utf-8")
        # use only letters (eliminate strange characters)
        tweet=" ".join(re.findall("[a-zA-Z]+", str(tweet)))
        
        analysis = TextBlob(tweet.strip())
        sentiment = analysis.sentiment
        dict_['polarity'] = sentiment.polarity         # evaluated polarity
        dict_['subjectivity'] = sentiment.subjectivity # evaluated subjectivity

    # Structure data in a pandas DataFrame with the results
    df = pd.DataFrame(dict_)
    
    # return (tweets mean polarity, tweets mean subjectivity)
    return (float(df.loc[:,"polarity"].mean()),float(df.loc[:,"subjectivity"].mean()))

Let's test out function with the 5th crypto in Top10:

In [6]:
print('* Testing Twitter Sentiments:')
crypto = GetTop10cryptos()[4]
print('-> Getting tweets and running sentiment analysis for',crypto,' ...')
pol, subj = GetTweetSentiments(crypto,language='en')
print('Polarity =', pol, 'Subjectivity =', subj)

* Testing Twitter Sentiments:
-> Getting tweets and running sentiment analysis for ('EOS', 'eos')  ...
Polarity = -0.05833333333333334 Subjectivity = 0.5


## Step 3: Entire code for Twiiter Analysis

You will use the previous functions to evaluate tweet's sentiments:
* First run SaveTwitterCredentials(file2save) to save your credentials. 
* Load the previous 2 functions: GetTop10cryptos() and GetTweetSentiments(crypto,language='en').

Now you can run the full code for tweets sentiment analysis:

In [7]:
import pandas as pd
import datetime

print('=> Twitter sentiment analysis using only 15 tweets ...')
print(datetime.datetime.today()) # print datatime of analysis

# create a list for statistics
stats = []

# analyse each crypto in Top10
for crypto in GetTop10cryptos():
    print('* ', crypto, '...')
    # get mean polarity and subjectivity for a crypto (using only 15 tweets)
    pol, subj = GetTweetSentiments(crypto,language='en')
    # add them to stats
    stats.append((crypto[0]+' OR '+crypto[1], pol, subj))

# create a dataframe for the mean values using the above list
df_stats = pd.DataFrame(stats, columns=['crypto','polarity','subjectivity'])

# Sort the results using polarity descending
df_stats.sort_values(by='polarity', inplace=True, ascending=False)  

# print means of the evaluations
print(df_stats)

print('\n* Max polarity:')
print(df_stats.loc[df_stats['polarity'].idxmax()])

print('\n* Max subjectivity:')
print(df_stats.loc[df_stats['subjectivity'].idxmax()])

print('Done!')

=> Twitter sentiment analysis using only 15 tweets ...
2018-11-03 21:14:00.584759
*  ('Bitcoin', 'bitcoin') ...
*  ('Ethereum', 'ethereum') ...
*  ('XRP', 'ripple') ...
*  ('Bitcoin Cash', 'bitcoin-cash') ...
*  ('EOS', 'eos') ...
*  ('Stellar', 'stellar') ...
*  ('Litecoin', 'litecoin') ...
*  ('Cardano', 'cardano') ...
*  ('Tether', 'tether') ...
*  ('Monero', 'monero') ...
                         crypto  polarity  subjectivity
3  Bitcoin Cash OR bitcoin-cash  0.250000      0.500000
1          Ethereum OR ethereum  0.200000      0.500000
6          Litecoin OR litecoin  0.137500      0.387500
0            Bitcoin OR bitcoin  0.000000      0.000000
2                 XRP OR ripple  0.000000      0.000000
8              Tether OR tether  0.000000      0.000000
9              Monero OR monero  0.000000      0.000000
4                    EOS OR eos -0.058333      0.500000
5            Stellar OR stellar -0.316667      0.422222
7            Cardano OR cardano       NaN           NaN

* Ma

## Acknowledgements

* https://stackabuse.com/accessing-the-twitter-api-with-python/
* https://www.youtube.com/watch?v=o_OZdbCzHUA&index=2&list=PL2-dafEMk2A6QKz1mrk1uIGfHkC1zZ6UU
* https://github.com/llSourcell/twitter_sentiment_challenge/blob/master/demo.py

### Hack for tweepy

If you decide to use *tweepy*, it will raise an error with python 3.7! So, you need to repace *async* variable with let's say *tr_async* in *\Lib\site-packages\streaming.py* (https://stackoverflow.com/questions/49339502/tweepy-on-macbook-pycharm-async-invalid-syntax).

Have fun!

2018@muntisa